In [1]:
#import os
import math
import fiona 
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Polygon, Point
from explode import explode

In [2]:
#SET COUNTRY <----------------------------------------------------------------------------------CHANGE
n = 'TUR' 

#SET CRS #<-------------------------------------------------------------------------------------CHANGE
crs = 'EPSG:5637' #turkey
#crs = 'EPSG:2100' #greece
wgs84= {'init' :'EPSG:4326'}

#open file
f = '../targets_2_simrdwn/' + n + '/'
file = f + 'predictions_' + n + '.csv'
detections = pd.read_csv(file, delimiter=',', header=0)
print(detections.shape)
detections.head()

#detections = detections[0:1000]

(13290, 8)


,img_file,img_width,img_height,label,xmin,ymin,xmax,ymax
0,image_18_38_46098_27_08505_38_47028_27_07399.jpeg,2061,2214,c,30,613,113,704
1,image_18_36_80047_28_24948_36_80986_28_2388.jpeg,1991,2186,c,1005,718,1101,891
2,image_18_38_57436_26_35616_38_5836_26_34498.jpeg,2084,2203,c,1005,1214,1100,1295
3,image_18_38_57436_26_35616_38_5836_26_34498.jpeg,2084,2203,c,1175,1366,1262,1466
4,image_18_38_57436_26_35616_38_5836_26_34498.jpeg,2084,2203,c,1181,1208,1261,1301


In [3]:
#georeference prediction bounding boxes and convert to centroid
selected = detections

lon_list, lat_list, radius_list, diameter_list = [], [], [], []
for i in range(len(selected)):
    #extract bottom right coordinate from image name
    img = detections['img_file'].iloc[i]
    br_lon = float(img.split('_')[4] + '.' + img.split('_')[5]) #lon = x                                       
    br_lat = float(img.split('_')[2] + '.' + img.split('_')[3]) #lat = y
    
    #extract top left coordinate from image name
    tl_lon = float(img.split('_')[8] + '.' + (img.split('_')[9]).split('.')[0]) #lon = x
    tl_lat = float(img.split('_')[6] + '.' + img.split('_')[7]) #lat = y    
    
#     #extract bottom right coordinate from image name
#     img = detections['img_file'].iloc[i]
#     br_lon = float(img.split('_')[8] + '.' + img.split('_')[9]) #lon = x                                       
#     br_lat = float(img.split('_')[6] + '.' + img.split('_')[7]) #lat = y
    
#     #extract top left coordinate from image name
#     tl_lon = float(img.split('_')[12] + '.' + (img.split('_')[13]).split('.')[0]) #lon = x
#     tl_lat = float(img.split('_')[10] + '.' + img.split('_')[11]) #lat = y
    
    #image dimensions
    img_w = detections['img_width'].iloc[i]
    img_h = detections['img_height'].iloc[i]
    width = abs(br_lon - tl_lon) 
    height = abs(tl_lat - br_lat)
    
    #decimal degrees per pixel
    res_x = width / img_w #image width
    res_y = height / img_h #image height
    
    #res
    lat = tl_lat * math.pi / 180
    res = 156543.04 * math.cos(lat) / (2 ** 18)
    
    #convert bounding box to centroid
    xmin = detections['xmin'].iloc[i]
    ymin = detections['ymin'].iloc[i]
    xmax = detections['xmax'].iloc[i]
    ymax = detections['ymax'].iloc[i]
    x = (xmin + xmax) / 2
    y = (ymin + ymax) / 2
    
    #convert centroid point to lat/lon   
    x_center = tl_lon + (x * res_x) 
    y_center = tl_lat - (y * res_y) 
    
    #estimate radius of detection
    w = xmax-xmin 
    h = ymax-ymin 
    radius = (((w+h)/2)/4 )*1.1
    diameter = radius * 2
    lon_list.append(x_center)
    lat_list.append(y_center)
    radius_list.append(radius)
    diameter_list.append(diameter)

In [4]:
#add attributes to df
d2 = selected.reset_index(drop=True) #super important
d2 = d2.drop(columns=['label'])
d2['x']=lon_list
d2['y']=lat_list
d2['radius']=np.round(radius_list,2)
d2['diameter']=np.round(diameter_list,2)
print(len(d2), 'total detections')

#add geometry information to df 
geometry = [Point(i) for i in zip(lon_list, lat_list)]
d2 = gpd.GeoDataFrame(d2, geometry=geometry, crs=wgs84)
d2 = d2.to_crs({'init': crs})

print(d2.crs)
d2.head()

13290 total detections
{'init': 'EPSG:5637'}


,img_file,img_width,img_height,xmin,ymin,xmax,ymax,x,y,radius,diameter,geometry
0,image_18_38_46098_27_08505_38_47028_27_07399.jpeg,2061,2214,30,613,113,704,27.074374,38.467514,23.92,47.85,POINT (5457336.632317584 1507076.414288142)
1,image_18_36_80047_28_24948_36_80986_28_2388.jpeg,1991,2186,1005,718,1101,891,28.244448,36.806404,36.99,73.98,POINT (5599828.388183808 1353999.777046428)
2,image_18_38_57436_26_35616_38_5836_26_34498.jpeg,2084,2203,1005,1214,1100,1295,26.350626,38.578338,24.20,48.40,POINT (5393933.816647355 1504941.096989041)
3,image_18_38_57436_26_35616_38_5836_26_34498.jpeg,2084,2203,1175,1366,1262,1466,26.351517,38.577661,25.71,51.42,POINT (5394024.773244331 1504885.534582943)
4,image_18_38_57436_26_35616_38_5836_26_34498.jpeg,2084,2203,1181,1208,1261,1301,26.351530,38.578338,23.79,47.58,POINT (5394009.618732309 1504958.149597286)


In [5]:
#select predictions within search area
search_area = gpd.read_file('../0_search_areas/4_search_area/search_area_100m_' + n + '.shp')
search_area = search_area.to_crs({'init': crs})

d2 = sjoin(d2, search_area, how='inner', op='within')
print(len(d2), 'detections within search area')

#select predictions by diameter
d = 55
d2 = d2[d2['diameter'] <= d]
geometry = [Point(i) for i in zip(d2['x'], d2['y'])]

d2 = d2[['radius', 'geometry']]
print(len(d2), 'total detections less than or equal to', d, 'meters')
d2.head()

10642 detections within search area
9650 total detections less than or equal to 55 meters


,radius,geometry
2,24.20,POINT (5393933.816647355 1504941.096989041)
3,25.71,POINT (5394024.773244331 1504885.534582943)
4,23.79,POINT (5394009.618732309 1504958.149597286)
5,24.34,POINT (5393993.114654925 1505030.460836815)
6,24.61,POINT (5393953.116187977 1504871.062021138)


In [6]:
#aggregate predictions 
buffer = gpd.GeoDataFrame(geometry = d2.buffer(10)) #buffer by 10 meters
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')
buffer_exploded = explode(buffer_dis)    
print(len(d2), 'predictions aggregated to', len(buffer_exploded), 'predictions')

9650 predictions aggregated to 2689 predictions


In [7]:
#calculate mean radius of aggregated predictions
d2_ag = gpd.sjoin(d2, buffer_exploded, how="inner", op='intersects')
centroids = gpd.GeoDataFrame(geometry = buffer_exploded.centroid, crs=wgs84)
centroids['radius']=d2_ag.groupby('index_right')['radius'].mean()
centroids.crs={'init' : crs}
print(len(centroids), 'predictions')
centroids.head()

C:\Users\Zack\Anaconda3\envs\thesis\lib\site-packages\geopandas\tools\sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


2689 predictions


,geometry,radius
0,POINT (5585256.743817649 1334762.655111267),14.020
1,POINT (5585402.066031021 1334864.4896085),13.200
2,POINT (5585343.659485435 1335108.628923445),10.105
3,POINT (5684442.50065573 1336382.959582468),16.500
4,POINT (5570861.722211564 1339524.890043325),14.710


In [8]:
#generate farmsite polygons
buffer = gpd.GeoDataFrame(geometry = centroids.buffer(100))
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')
buffer_exploded = explode(buffer_dis)    
buffer_exploded.crs={'init' : crs}
print(len(buffer_exploded), 'total farmsite predictions')
buffer_exploded.head()

333 total farmsite predictions


,geometry
0,"POLYGON ((5585356.743817649 1334762.655111267,..."
1,"POLYGON ((5585443.659485435 1335108.628923445,..."
2,"POLYGON ((5684542.50065573 1336382.959582468, ..."
3,"POLYGON ((5570961.722211564 1339524.890043325,..."
4,"POLYGON ((5683196.438575359 1341022.169360989,..."


In [9]:
#add number of predictions per farmsite to farmsite polygons
count=sjoin(centroids, buffer_exploded, how='inner', op='within')
buffer_exploded['count']=count.groupby('index_right')['index_right'].count()
buffer_exploded["farm_id"] = buffer_exploded.index + 1 #add farm ID
buffer_exploded.head()

,geometry,count,farm_id
0,"POLYGON ((5585356.743817649 1334762.655111267,...",2,1
1,"POLYGON ((5585443.659485435 1335108.628923445,...",1,2
2,"POLYGON ((5684542.50065573 1336382.959582468, ...",1,3
3,"POLYGON ((5570961.722211564 1339524.890043325,...",1,4
4,"POLYGON ((5683196.438575359 1341022.169360989,...",1,5


In [10]:
#select farmsites by number of predictions
p = 4 #number of predictions per farmsite
farmsites = buffer_exploded[buffer_exploded['count'] >= p]
print(len(farmsites), 'farmsite predictions')
farmsites.head()

128 farmsite predictions


,geometry,count,farm_id
21,"POLYGON ((5631987.630490826 1361535.366033874,...",9,22
25,"POLYGON ((5804822.691990278 1365594.318564056,...",9,26
27,"POLYGON ((5524092.456876466 1371532.797600841,...",13,28
28,"POLYGON ((5631280.445168911 1361865.198807753,...",13,29
29,"POLYGON ((5632006.905733245 1361203.483044021,...",5,30


In [11]:
#select predictions by farmsites
select = farmsites
select['predictions'] = 0
select = select.dissolve(by='predictions')
mask = centroids.within(select.loc[0, 'geometry'])
farmsites=farmsites.drop(['predictions'], axis=1)
d2_clip = centroids.loc[mask]

print('detections within farmsites:', len(d2_clip))
print(d2_clip.crs)
d2_clip.head()

C:\Users\Zack\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


detections within farmsites: 2418
{'init': 'EPSG:5637'}


,geometry,radius
21,POINT (5633117.398942553 1360647.30999176),7.70
23,POINT (5633172.121144095 1360700.05971877),16.36
25,POINT (5633033.17917203 1360683.82139695),11.55
26,POINT (5632826.544772988 1360686.588985316),17.19
27,POINT (5633188.910969399 1360772.329830664),11.96


In [12]:
#add id and farm_id to predictions
d2_clip_id = sjoin(farmsites, d2_clip, how='right', op='intersects')
d2_clip_id=d2_clip_id.reset_index(drop=True)
d2_clip_id["id"] = d2_clip_id.index + 1 
d2_clip_id['diameter']=d2_clip_id['radius']*2
d2_clip=d2_clip_id[['id', 'farm_id', 'radius', 'diameter', 'geometry']]
print('detections:', len(d2_clip_id))
d2_clip.head()

detections: 2418


,id,farm_id,radius,diameter,geometry
0,1,22,10.040000,20.080000,POINT (5631887.630490826 1361535.366033874)
1,2,22,9.626667,19.253333,POINT (5631845.293616918 1361599.681664578)
2,3,22,7.290000,14.580000,POINT (5631699.55357015 1361605.920009058)
3,4,22,8.940000,17.880000,POINT (5631793.236179897 1361637.75549508)
4,5,22,11.820000,23.640000,POINT (5631827.857963587 1361640.558857492)


In [13]:
#generate farmsite extents
envelope = gpd.GeoDataFrame(geometry = farmsites.envelope)
farmsites2=farmsites.copy()
farmsites2['geometry'] = envelope['geometry']
farmsites2.head()

,geometry,count,farm_id
21,"POLYGON ((5631558.676879461 1361435.366033874,...",9,22
25,"POLYGON ((5804590.57047427 1365513.892462137, ...",9,26
27,"POLYGON ((5523811.518937072 1371432.797600841,...",13,28
28,"POLYGON ((5630767.091081615 1361447.499086719,...",13,29
29,"POLYGON ((5631808.132107622 1361118.306108944,...",5,30


In [14]:
path= f + n + '_exclude.kml'
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
exclude = gpd.read_file(path, driver='KML')
exclude = exclude.to_crs({'init': crs})
print(exclude.crs)

exclude['Dissolve'] = 0
exclude_dis = exclude.dissolve(by='Dissolve')
mask = ~d2_clip.within(exclude_dis.loc[0, 'geometry'])
d2_clip = d2_clip.loc[mask]
d2_clip.head()

{'init': 'EPSG:5637'}


,id,farm_id,radius,diameter,geometry
9,10,26,7.77,15.54,POINT (5804724.629710294 1365613.892462137)
10,11,26,7.70,15.40,POINT (5804709.543797413 1365638.928925416)
11,12,26,8.66,17.32,POINT (5804737.988619858 1365647.278875503)
12,13,26,8.89,17.78,POINT (5804716.73346681 1365675.660826551)
13,14,26,8.11,16.22,POINT (5804697.641777741 1365698.166477813)


In [15]:
#generate net pen buffers
def buffer(row):
     return row.geometry.buffer(row.radius)   
    
d2_clip2 = d2_clip.copy()
d2_clip2.crs

buff = d2_clip2['geometry'] = d2_clip2.apply(buffer, axis=1)
circles = gpd.GeoDataFrame(d2_clip2, geometry = buff, crs={'init': crs})

#export buffered centroids
circles.to_file(f + n + '_simrdwn_pens.shp')
circles['geometry'] = circles['geometry'].to_crs(epsg=4326)
circles.to_file(f + n + '_simrdwn_pens.geojson', driver='GeoJSON')

In [16]:
#export centroids
d2 = d2.to_crs({'init': crs})
d2.to_file(f + n + '_simrdwn.shp')
d2['geometry'] = d2['geometry'].to_crs(epsg=4326)
d2.to_file(f + n + '_simrdwn.geojson', driver='GeoJSON')

#export centroids within farmsites
d2_clip.to_file(f + n + '_simrdwn_pts.shp')
d2_clip['geometry'] = d2_clip['geometry'].to_crs(epsg=4326)
d2_clip.to_file(f + n + '_simrdwn_pts.geojson', driver='GeoJSON')

#export farmsites
# farmsites.to_file(f + n + '_simrdwn_farmsites.shp')
# farmsites['geometry'] = farmsites['geometry'].to_crs(epsg=4326)
# farmsites.to_file(f + n + '_simrdwn_farmsites.geojson', driver='GeoJSON')

#export farmsites extents
farmsites2.to_file(f + n + '_simrdwn_farmsites_ext.shp')
farmsites2['geometry'] = farmsites2['geometry'].to_crs(epsg=4326)
farmsites2.to_file(f + n + '_simrdwn_farmsites_ext.geojson', driver='GeoJSON')